In [ ]:
from docx import Document

def is_heading(paragraph):
    """Checks if a paragraph is a heading.

    :param paragraph:
    :return:
    """
    if paragraph.style.name=='Heading':
        return True


def iterate_document_sections(document):
    """For each headed section, generate a sequence of paragraphs.
    Each sequence starts with a headed paragraph, followed by text paragraphs.

    :param document: Current Comet Issue
    :return: an article
    """
    paragraphs = [document.paragraphs[0]]
    for paragraph in document.paragraphs[1:]:
        if is_heading(paragraph):
            yield paragraphs
            paragraphs = [paragraph]
            continue
        paragraphs.append(paragraph)
    yield paragraphs


def create_document_from_paragraphs(paragraphs):
    """Iterates through the paragraphs containing articles and splits them into separate files.

    :param paragraphs: Article text
    :return: New document with a single article.
    """
    new_doc = Document()
    n = 0
    for count, words in enumerate(paragraphs):
        n += 1
        new_content = words.text
        new_doc.add_paragraph(new_content)
        print (new_content)
    #new_doc.save('new' + str(n) + '.docx')
    return new_doc
    
document = docx.Document('/Users/oskar/Python/small_projects/Syndromes_Flash_Cards/Essential genetic syndromes.docx')
n = 0
for paragraphs in iterate_document_sections(document):
    n +=1 
    create_document_from_paragraphs(paragraphs).save('separated/new' + str(n) + '.docx')

In [46]:
from docx import Document
import re
import pandas as pd
import glob
syndrome_name = []
information_all = []
for name in glob.glob('/Users/oskar/Python/small_projects/Syndromes_Flash_Cards/separated/*.docx'):
    document = Document(name)
    syndrome_name.append(document.paragraphs[0].text)
    information = []
    for para in document.paragraphs[1:]:
        information.append((para.text))
    information_all.append(information)
len(syndrome_name)

162

In [ ]:
syndrome_name.sort()
syndrome_name

In [ ]:
for name, element in zip(syndrome_name,information_final):
    print(name, len(element))

In [61]:
information_final = []
for element in information_all:
    information_final.append(re.split('Genetics, |Clinical findings/Dysmorphic features, |Etiology, |Pathogenesis, |Genetic testing/diagnosis, |Others,',', '.join(element)))

column_names = ['empty','Genetics','Clinical findings/Dysmorphic features','Etiology', 'Pathogenesis','Genetic testing/diagnosis','Others']
df = pd.DataFrame(information_final, columns=column_names)
df.drop('empty', axis=1, inplace=True)
df['Syndrome name'] = syndrome_name

#import figures links
df_figures = pd.read_csv('category_wordcloud_figure.csv', sep=';',usecols=['Category','png links','figure'])

df_final = pd.concat([df, df_figures], axis=1, join='inner')
df_final.to_csv('df_for_app.csv')